In [ ]:
import json

dataset_path = "/content/drive/MyDrive/Bulipe/game_dataset.jsonl"

data = []
with open(dataset_path, "r") as f:
    for line in f:
        item = json.loads(line)
        data.append((item["prompt"], item["code"]))


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")


prompts = [item[0] for item in data]

embeddings = model.encode(prompts, convert_to_tensor=True, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import torch
from torch.nn.functional import cosine_similarity

def find_best_match(user_input, top_k=1):
    user_embedding = model.encode(user_input, convert_to_tensor=True)
    scores = cosine_similarity(user_embedding, embeddings)
    top_k_indices = torch.topk(scores, k=top_k).indices

    results = []
    for idx in top_k_indices:
        prompt, completion = data[idx]
        similarity = scores[idx].item()
        results.append({
            "prompt": prompt,
            "completion": completion,
            "similarity": similarity
        })
    return results


In [ ]:
user_input = "Build a Pong game with two paddles and a bouncing ball "
matches = find_best_match(user_input)

for m in matches:
    print(f"Prompt: {m['prompt']}")
    print(f"Completion: {m['completion']}")
    print(f"Similarity Score: {m['similarity']:.4f}")
    print("-" * 50)


Prompt: Build a Pong game with two paddles and a bouncing ball using JavaScript.
Completion: <!DOCTYPE html>
<html>
<head>
  <title>Pong Game</title>
  <style>
    canvas { background: black; display: block; margin: 0 auto; }
  </style>
</head>
<body>
<canvas id="pong" width="600" height="400"></canvas>
<script>
  const canvas = document.getElementById("pong");
  const ctx = canvas.getContext("2d");
  const paddleWidth = 10, paddleHeight = 100;
  let leftY = 150, rightY = 150, ballX = 300, ballY = 200, ballDX = 5, ballDY = 5;

  function drawRect(x, y, w, h, color) {
    ctx.fillStyle = color;
    ctx.fillRect(x, y, w, h);
  }

  function drawCircle(x, y, r, color) {
    ctx.fillStyle = color;
    ctx.beginPath();
    ctx.arc(x, y, r, 0, Math.PI * 2);
    ctx.closePath();
    ctx.fill();
  }

  document.addEventListener('keydown', (e) => {
    if (e.key === 'w') leftY -= 20;
    if (e.key === 's') leftY += 20;
    if (e.key === 'ArrowUp') rightY -= 20;
    if (e.key === 'ArrowDown') ri